### Usando a implementação do modelo Word2Vec da biblioteca: gensim

In [ ]:
!pip install gensim==3.8.3

     |████████████████████████████████| 24.2 MB 1.8 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import pandas as pd
import gensim as gensim
import numpy as np
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.scripts import word2vec2tensor
from gensim.models.callbacks import CallbackAny2Vec
import re
from google.colab import drive

### Esse notebook Jupyter foi criado usando Google Colab. Essa linha apenas monta a sua pasta do drive para salvar os arquivos dos modelos gerados em uma pasta que você pode definir o nome nas linhas abaixo (TCCLG nesse caso)

In [ ]:
drive.mount('/content/drive')
%cd /content/drive/My Drive/TCCLG

Mounted at /content/drive
/content/drive/My Drive/TCCLG


### Essa parte do código carrega o csv das diferentes representações das palavras de um dicionário e faz um pequeno parsing, construindo também o conjunto de palavras que vão ser incorporadas nesse modelo

In [ ]:
df = pd.read_csv('etiquetas2.csv')
df = df.replace(':','', regex=True)
print(df.head())
restricted_word_set = df['palavra'].to_numpy()

      palavra  ... Unnamed: 4
0         aba  ...        NaN
1       ababa  ...        NaN
2      abaçai  ...        NaN
3      abaças  ...        NaN
4  abacateiro  ...        NaN

[5 rows x 5 columns]


### A função abaixo serve para restringir um "vocabulário" do word2vec. Nesse contexto, utilizamos isto para remover as silabas individuais geradas mais a frente para que o modelo retorne apenas palavras completas. Isso fica mais claro mais a frente.

In [ ]:
# AVISO: quando você cria o modelo pela primeira vez, a propriedade vectors_norm é igual a None.
# Essa propriedade só é atualizada com o valor dos vetores após o primeiro uso, no formato numpy.ndarray.
# Dessa forma, antes de usar a função abaixo, é necessário utilizar o modelo uma primeira vez para
# inicializar os valores. Basta chamar algo como most_similar("gato").
def restrict_w2v(w2v, restricted_word_set):
    new_vectors = []
    new_vocab = {}
    new_index2entity = []
    new_vectors_norm = []

    for i in range(len(w2v.wv.vocab)):
        word = w2v.wv.index2entity[i]
        vec = w2v.wv.vectors[i]
        vocab = w2v.wv.vocab[word]
        vec_norm = w2v.wv.vectors_norm[i]
        if word in restricted_word_set:
            vocab.index = len(new_index2entity)
            new_index2entity.append(word)
            new_vocab[word] = vocab
            new_vectors.append(vec)
            new_vectors_norm.append(vec_norm)

    w2v.wv.vocab = new_vocab
    w2v.wv.vectors = np.array(new_vectors)
    w2v.wv.index2entity = new_index2entity
    w2v.wv.index2word = new_index2entity
    w2v.wv.vectors_norm = new_vectors_norm

### Isso é apenas uma classe de logger para que consigamos ver o progresso do treinamento do modelo. Isso é um parâmetro da função, passado como callback

In [ ]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

        
epoch_logger = EpochLogger()

# Modelos utilizando representação silábica
---



### Modelo básico (Frases Simples):
#### Nesse modelo, cada "frase'' do corpo de texto é dada por uma palavra original do vocabulário e uma das representações dividida em silabas. Exemplo: ["abacaxi", "a", "ba", "ca", "'xi"]

In [ ]:
df_silabas = df[['palavra', 'silabas']]
print(df_silabas.head())
print("")

      palavra          silabas
0         aba            'a.ba
1       ababa         a.'ba.ba
2      abaçai       a.ba.ça.'i
3      abaças        a.'ba.ças
4  abacateiro  a.ba.ca.'tei.ro



#### O começo da "frase" é a palavra original e as "silabas" são concatenadas na frase para que similaridades possam ser encontradas em um nível silábico pelo Word2Vec. A ideia aqui é que palavras com silabas semelhantes devem ter um alto nível de similaridade pelo modelo. Como as silabas aparecendo "proximas" das palavras na frase, essa semelhança é identificada até certo ponto pelo Word2Vec.

In [ ]:
corpus_silabas_basico = []
for index, row in df_silabas.iterrows():
  sentence = [row['palavra']]
  for syllable in row['silabas'].split('.'):
    sentence.append(syllable)
  corpus_silabas_basico.append(sentence)

In [ ]:
for k, v in enumerate(corpus_silabas_basico[:2]):
  print(f'{k}: {v}')

0: ['aba', "'a", 'ba']
1: ['ababa', 'a', "'ba", 'ba']


#### Após montar a representação, treinamos o modelo com os parâmetros definidos abaixo após multiplos testes. O modelo foi treinado por 30 iterações.

In [ ]:
model_silabas_basico = Word2Vec(corpus_silabas_basico, min_count=1, workers=8, window=10, sg=1, size=300, iter=30, callbacks=[epoch_logger])

# AVISO: esse é o print de primeiro uso mencionado anteriormente para inicializar os valores necessários para a função restrict_w2v
print(model_silabas_basico.wv.most_similar('abaçai'))

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
Epoch #10 start
Epoch #10 end
Epoch #11 start
Epoch #11 end
Epoch #12 start
Epoch #12 end
Epoch #13 start
Epoch #13 end
Epoch #14 start
Epoch #14 end
Epoch #15 start
Epoch #15 end
Epoch #16 start
Epoch #16 end
Epoch #17 start
Epoch #17 end
Epoch #18 start
Epoch #18 end
Epoch #19 start
Epoch #19 end
Epoch #20 start
Epoch #20 end
Epoch #21 start
Epoch #21 end
Epoch #22 start
Epoch #22 end
Epoch #23 start
Epoch #23 end
Epoch #24 start
Epoch #24 end
Epoch #25 start
Epoch #25 end
Epoch #26 start
Epoch #26 end
Epoch #27 start
Epoch #27 end
Epoch #28 start
Epoch #28 end
Epoch #29 start
Epoch #29 end
[('abaiba', 0.9400215148925781), ('açai', 0.9350585341453552), ('buiça', 0.9310318231582642), ('abaira', 0.9283071756362

#### Por fim, restringimos o vocabulário do modelo para remover as silabas. Dessa forma, as semelhanças encontradas são apenas com as palavras completas das frases geradas acima.

In [ ]:
restrict_w2v(model_silabas_basico, restricted_word_set)

#### Alguns exemplos dos resultados obtidos de coeficientes de similaridades para diferentes palavras. É facil notar a similaridade de algumas destas:

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_silabas_basico.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_silabas_basico.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to bantu:')
for k, v in enumerate(model_silabas_basico.wv.most_similar('bantu')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
jaguara: 0.9316715002059937
guajara: 0.9187493324279785
aguarras: 0.9095985889434814
jaguaquara: 0.9095667600631714
caraguata: 0.9082787036895752

Most similar to gue:
morreira: 0.5855243802070618
guerrilheira: 0.5770751237869263
carreia: 0.5731767416000366
carreira: 0.5655220746994019
parreiras: 0.5650065541267395

Most similar to bantu:
matura: 0.9224377870559692
tortura: 0.9207016825675964
catu: 0.9203349947929382
montu: 0.9194598197937012
soltura: 0.9154787063598633


#### Essa linha salva o modelo na pasta local. Caso esteja usando o colab, salvará na pasta definida nas primeiras linhas desse documento.

In [ ]:
model_silabas_basico.wv.save_word2vec_format("model_silabas_basico.model", binary=True)

#### Essa linha salva o arquivo do formato tensorflow que pode ser utilizado com o modelo acima em https://projector.tensorflow.org/ para uma visualização dos dados em forma de gráfico 3D com clusters de palavras similares.

In [ ]:
word2vec2tensor.word2vec2tensor("model_silabas_basico.model", "tensor-model-silabas-basico", True)

### Modelo Contagem (Frases com contador relativo a silaba)
#### Nesse modelo, cada frase do corpo de texto é dada por uma palavra original do vocabulário e uma das representações dividida em silabas, com silabas duplicadas sendo diferenciadas por um contador utilizado como prefixo. Essa estratégia garante que o modelo não seja influenciado no aprendizado a acreditar que palavras que possuem silabas similares duplicadas possuem um grau de similaridade maior. Exemplo: ["anarcocomunismo", "0-a", "0-nar", "0-co", "1-co", "0-mu", "0-'nis", "0-mo"]


In [ ]:
corpus_silabas_contagem = []
for index, row in df_silabas.iterrows():
  sentence = [row['palavra']]
  syllables = []
  for syllable in row['silabas'].split('.'):
    modified_syllable = f'{syllables.count(syllable)}-{syllable}'
    syllables.append(syllable)
    sentence.append(modified_syllable)
  corpus_silabas_contagem.append(sentence)

In [ ]:
for k, v in enumerate(corpus_silabas_contagem[:3]):
  print(f'{k}: {v}')

0: ['aba', "0-'a", '0-ba']
1: ['ababa', '0-a', "0-'ba", '0-ba']
2: ['abaçai', '0-a', '0-ba', '0-ça', "0-'i"]


In [ ]:
model_silabas_contagem = Word2Vec(corpus_silabas_contagem, min_count=1, workers=8, window=30, sg=1, size=300, iter=30, callbacks=[epoch_logger])
print(model_silabas_contagem.wv.most_similar('abaçai'))

Epoch #30 start
Epoch #30 end
Epoch #31 start
Epoch #31 end
Epoch #32 start
Epoch #32 end
Epoch #33 start
Epoch #33 end
Epoch #34 start
Epoch #34 end
Epoch #35 start
Epoch #35 end
Epoch #36 start
Epoch #36 end
Epoch #37 start
Epoch #37 end
Epoch #38 start
Epoch #38 end
Epoch #39 start
Epoch #39 end
Epoch #40 start
Epoch #40 end
Epoch #41 start
Epoch #41 end
Epoch #42 start
Epoch #42 end
Epoch #43 start
Epoch #43 end
Epoch #44 start
Epoch #44 end
Epoch #45 start
Epoch #45 end
Epoch #46 start
Epoch #46 end
Epoch #47 start
Epoch #47 end
Epoch #48 start
Epoch #48 end
Epoch #49 start
Epoch #49 end
Epoch #50 start
Epoch #50 end
Epoch #51 start
Epoch #51 end
Epoch #52 start
Epoch #52 end
Epoch #53 start
Epoch #53 end
Epoch #54 start
Epoch #54 end
Epoch #55 start
Epoch #55 end
Epoch #56 start
Epoch #56 end
Epoch #57 start
Epoch #57 end
Epoch #58 start
Epoch #58 end
Epoch #59 start
Epoch #59 end
[('açai', 0.9425680041313171), ('buiça', 0.9326292276382446), ('jaiba', 0.9273122549057007), ('abair

In [ ]:
restrict_w2v(model_silabas_contagem, restricted_word_set)

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_silabas_contagem.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_silabas_contagem.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to grisalha:')
for k, v in enumerate(model_silabas_contagem.wv.most_similar('grisalha')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to sintetizado:')
for k, v in enumerate(model_silabas_contagem.wv.most_similar('sintetizado')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
jupia-agua: 0.9159449338912964
jaguara: 0.9101862907409668
pixeis: 0.9063030481338501
prorpia: 0.9011303186416626
guarapiranga: 0.8986994624137878

Most similar to gue:
guera: 0.9509106874465942
ague: 0.9470062255859375
guegue: 0.9171435832977295
guede: 0.9151749610900879
gongue: 0.9076242446899414

Most similar to grisalha:
grisalho: 0.9342557191848755
grisalhos: 0.9277743101119995
sadao: 0.9002761840820312
desaguam: 0.8994162082672119
rosada: 0.8992834091186523

Most similar to sintetizado:
sintetizada: 0.9840803146362305
sintetizadas: 0.9819157123565674
sintetizados: 0.9809841513633728
sintetizava: 0.9753881692886353
sintetizar: 0.9724298119544983


In [ ]:
model_silabas_contagem.wv.save_word2vec_format("model_silabas_contagem.model", binary=True)

In [ ]:
word2vec2tensor.word2vec2tensor("model_silabas_contagem.model", "tensor-model-silabas-contagem", True)

### Modelo Chunks (frases com sub-palavras)
#### Nesse modelo, cada frase do corpo de texto é dada por uma palavra original do vocabulário e uma das representações dividida em silabas. Além disso, são geradas todas sub-palavras possíveis com as silabas levando em consideração a ordem das mesmas na palavra original. Essa estratégia tem como objetivo tentar influenciar o aprendizado a detectar similaridade de contexto com base em sub-palavras que podem ocorrer dentro de palavras maiores, similarmente a estrategia do FastText mas considerando silabas ao invés de n-grams. Exemplo: ["abacaxi", "a", "ba", "ca", "'xi", "aba", "baca", "ca'xi", "abaca", "baca'xi", "abaca'xi"]


In [ ]:
corpus_silabas_chunk = []
for index, row in df_silabas.iterrows():
  sentence = [row['palavra']]
  syllables = []
  modified_syllables = []
  chunks = []
  for syllable in row['silabas'].split('.'):
    modified_syllable = f'{syllable}'
    syllables.append(syllable)
    modified_syllables.append(modified_syllable)
    sentence.append(modified_syllable)
  for i in range(2, len(modified_syllables)+1):
    for j in range(0, len(modified_syllables)):
      start = j
      end = j + i
      if (end <= len(modified_syllables)):
        chunk = ' '.join(syllables[start:end])
        sentence.append(f"{chunk}")
        chunks.append(chunk)
  corpus_silabas_chunk.append(sentence)

In [ ]:
for k, v in enumerate(corpus_silabas_chunk[:3]):
  print(f'{k}: {v}')

0: ['aba', "'a", 'ba', "'a ba"]
1: ['ababa', 'a', "'ba", 'ba', "a 'ba", "'ba ba", "a 'ba ba"]
2: ['abaçai', 'a', 'ba', 'ça', "'i", 'a ba', 'ba ça', "ça 'i", 'a ba ça', "ba ça 'i", "a ba ça 'i"]


In [ ]:
model_silabas_chunk = Word2Vec(corpus_silabas_chunk, min_count=1, workers=8, window=50, sg=1, size=300, iter=30, callbacks=[epoch_logger])
print(model_silabas_chunk.wv.most_similar('abaçai'))

Epoch #60 start
Epoch #60 end
Epoch #61 start
Epoch #61 end
Epoch #62 start
Epoch #62 end
Epoch #63 start
Epoch #63 end
Epoch #64 start
Epoch #64 end
Epoch #65 start
Epoch #65 end
Epoch #66 start
Epoch #66 end
Epoch #67 start
Epoch #67 end
Epoch #68 start
Epoch #68 end
Epoch #69 start
Epoch #69 end
Epoch #70 start
Epoch #70 end
Epoch #71 start
Epoch #71 end
Epoch #72 start
Epoch #72 end
Epoch #73 start
Epoch #73 end
Epoch #74 start
Epoch #74 end
Epoch #75 start
Epoch #75 end
Epoch #76 start
Epoch #76 end
Epoch #77 start
Epoch #77 end
Epoch #78 start
Epoch #78 end
Epoch #79 start
Epoch #79 end
Epoch #80 start
Epoch #80 end
Epoch #81 start
Epoch #81 end
Epoch #82 start
Epoch #82 end
Epoch #83 start
Epoch #83 end
Epoch #84 start
Epoch #84 end
Epoch #85 start
Epoch #85 end
Epoch #86 start
Epoch #86 end
Epoch #87 start
Epoch #87 end
Epoch #88 start
Epoch #88 end
Epoch #89 start
Epoch #89 end
[("a ba ça 'i", 0.9919774532318115), ("ba ça 'i", 0.991045355796814), ('a ba ça', 0.990084171295166)

In [ ]:
restrict_w2v(model_silabas_chunk, restricted_word_set)

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_silabas_chunk.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_silabas_chunk.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to grisalha:')
for k, v in enumerate(model_silabas_chunk.wv.most_similar('grisalha')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to sintetizado:')
for k, v in enumerate(model_silabas_chunk.wv.most_similar('sintetizado')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to aba:')
for k, v in enumerate(model_silabas_chunk.wv.most_similar('aba')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bata:')
for k, v in enumerate(model_silabas_chunk.wv.most_similar('bata')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bantu:')
for k, v in enumerate(model_silabas_chunk.wv.most_similar('bantu')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
capita: 0.8753652572631836
apitado: 0.8666909337043762
capitana: 0.8663016557693481
capitado: 0.8657164573669434
piata: 0.8624604940414429

Most similar to gue:
guedaz: 0.6425230503082275
gueonim: 0.6158539056777954
guetim: 0.5770539045333862
agueda: 0.5761498212814331
foguetao: 0.5741961598396301

Most similar to grisalha:
grisalhos: 0.9111911654472351
grisalho: 0.9025635719299316
sasa: 0.832282304763794
sasanha: 0.826266348361969
desaba: 0.814529538154602

Most similar to sintetizado:
sintetizada: 0.975807785987854
sintetizados: 0.9715960621833801
sintetizar: 0.9694432020187378
sintetizadas: 0.9678192734718323
sintetizou: 0.9615318775177002

Most similar to aba:
piaba: 0.9283735752105713
goiaba: 0.9027817249298096
diaba: 0.9016405344009399
abacos: 0.8900822997093201
abalos: 0.8804158568382263

Most similar to bata:
taba: 0.9391489028930664
batava: 0.9265483617782593
batalham: 0.91791170835495
estaba: 0.9079567193984985
batalha: 0.9055522084236145

Most simi

In [ ]:
model_silabas_chunk.wv.save_word2vec_format("model_silabas_chunk.model", binary=True)

In [ ]:
word2vec2tensor.word2vec2tensor("model_silabas_chunk.model", "tensor-silabas-chunk", True)

### Contagem + Chunks (frases com sub-palavras e contador)
#### Nesse modelo, cada frase do corpo de texto é dada por uma palavra original do vocabulário e uma das representações dividida em silabas,  com silabas duplicadas sendo diferenciadas por um contador utilizado como prefixo. Além disso, são geradas todas sub-palavras possíveis com as silabas levando em consideração a ordem das mesmas na palavra original. As sub-palavras também possuem um prefixo de contador de número de ocorrências. O objetivo é combinar as duas estratégias anteriores e obter as vantagens de ambas estratégias. Exemplo: ["abacaxi", "0-a", "0-ba", "0-ca", "0-'xi", "0\_aba", "0\_baca", "0\_ca'xi", "0\_abaca", "0\_baca'xi", "0\_abaca'xi"]

In [ ]:
corpus_silabas_contagem_chunk = []
for index, row in df_silabas.iterrows():
  sentence = [row['palavra']]
  syllables = []
  modified_syllables = []
  chunks = []
  for syllable in row['silabas'].split('.'):
    modified_syllable = f'{syllables.count(syllable)}-{syllable}'
    syllables.append(syllable)
    modified_syllables.append(modified_syllable)
    sentence.append(modified_syllable)
  for i in range(2, len(modified_syllables)+1):
    for j in range(0, len(modified_syllables)):
      start = j
      end = j + i
      if (end <= len(modified_syllables)):
        chunk = ' '.join(syllables[start:end])
        sentence.append(f"{chunks.count(chunk)}_{chunk}")
        chunks.append(chunk)
  corpus_silabas_contagem_chunk.append(sentence)

In [ ]:
for k, v in enumerate(corpus_silabas_contagem_chunk[:3]):
  print(f'{k}: {v}')

0: ['aba', "0-'a", '0-ba', "0_'a ba"]
1: ['ababa', '0-a', "0-'ba", '0-ba', "0_a 'ba", "0_'ba ba", "0_a 'ba ba"]
2: ['abaçai', '0-a', '0-ba', '0-ça', "0-'i", '0_a ba', '0_ba ça', "0_ça 'i", '0_a ba ça', "0_ba ça 'i", "0_a ba ça 'i"]


In [ ]:
model_silabas_contagem_chunk = Word2Vec(corpus_silabas_contagem_chunk, min_count=1, workers=8, window=50, sg=1, size=300, iter=30, callbacks=[epoch_logger])
print(model_silabas_contagem_chunk.wv.most_similar('abaçai'))

Epoch #90 start
Epoch #90 end
Epoch #91 start
Epoch #91 end
Epoch #92 start
Epoch #92 end
Epoch #93 start
Epoch #93 end
Epoch #94 start
Epoch #94 end
Epoch #95 start
Epoch #95 end
Epoch #96 start
Epoch #96 end
Epoch #97 start
Epoch #97 end
Epoch #98 start
Epoch #98 end
Epoch #99 start
Epoch #99 end
Epoch #100 start
Epoch #100 end
Epoch #101 start
Epoch #101 end
Epoch #102 start
Epoch #102 end
Epoch #103 start
Epoch #103 end
Epoch #104 start
Epoch #104 end
Epoch #105 start
Epoch #105 end
Epoch #106 start
Epoch #106 end
Epoch #107 start
Epoch #107 end
Epoch #108 start
Epoch #108 end
Epoch #109 start
Epoch #109 end
Epoch #110 start
Epoch #110 end
Epoch #111 start
Epoch #111 end
Epoch #112 start
Epoch #112 end
Epoch #113 start
Epoch #113 end
Epoch #114 start
Epoch #114 end
Epoch #115 start
Epoch #115 end
Epoch #116 start
Epoch #116 end
Epoch #117 start
Epoch #117 end
Epoch #118 start
Epoch #118 end
Epoch #119 start
Epoch #119 end
[('0_a ba ça', 0.9888284206390381), ("0_a ba ça 'i", 0.98838

In [ ]:
restrict_w2v(model_silabas_contagem_chunk, restricted_word_set)

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_silabas_contagem_chunk.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_silabas_contagem_chunk.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to grisalha:')
for k, v in enumerate(model_silabas_contagem_chunk.wv.most_similar('grisalha')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to sintetizado:')
for k, v in enumerate(model_silabas_contagem_chunk.wv.most_similar('sintetizado')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to aba:')
for k, v in enumerate(model_silabas_contagem_chunk.wv.most_similar('aba')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bata:')
for k, v in enumerate(model_silabas_contagem_chunk.wv.most_similar('bata')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bantu:')
for k, v in enumerate(model_silabas_contagem_chunk.wv.most_similar('bantu')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
capiata: 0.8657286167144775
capita: 0.860567033290863
guarita: 0.8540998697280884
apitadas: 0.8532338738441467
capitana: 0.8501086831092834

Most similar to gue:
ague: 0.9556980729103088
guegue: 0.8959865570068359
guede: 0.895412802696228
guerra: 0.8949964046478271
guera: 0.8938165903091431

Most similar to grisalha:
grisalhos: 0.9251331090927124
grisalho: 0.9205228090286255
sasa: 0.847955584526062
desaba: 0.8197441697120667
sacha: 0.8177820444107056

Most similar to sintetizado:
sintetizada: 0.9715463519096375
sintetizados: 0.9674364924430847
sintetizar: 0.9610275030136108
sintetizadas: 0.9598528146743774
sintetizando: 0.9593729972839355

Most similar to aba:
piaba: 0.9184836745262146
diaba: 0.9038135409355164
goiaba: 0.890017569065094
abacos: 0.8872066736221313
abalos: 0.8814542293548584

Most similar to bata:
taba: 0.9478793740272522
batava: 0.9324724674224854
estaba: 0.9224439859390259
batalha: 0.9104498028755188
contaba: 0.8947254419326782

Most similar 

In [ ]:
model_silabas_contagem_chunk.wv.save_word2vec_format("model_silabas_contagem_chunk.model", binary=True)

In [ ]:
word2vec2tensor.word2vec2tensor("model_silabas_contagem_chunk.model", "tensor-silabas-contagem-chunk", True)

### FastText - Contagem + Chunks
#### Esse modelo acabou sendo discutido apenas brevemente no TCC, mas é a mesma representação acima sendo utilizada para treinar outro modelo, o FastText, que diferentemente do Word2Vec, tem por padrão a funcionalidade de identificar similaridades em silabas sem necessidade da inclusão delas na "frase". Apesar disso, a detecção de silabas do FastText é implementada pela utilização de n-grams, que são pequenas parcelas da palavra de tamanhos variados. Isso não é bem preciso para as nossas representações de etiquetagem e de fonética, visto que existem símbolos que quando removidos tornam completamente diferente a representação da palavra. Apesar disso, foi decidido tentar testar os resultados obtidos com esse outro modelo para comparação com o Word2Vec.

In [ ]:
corpus_silabas_fast_text_cc = []
for index, row in df_silabas.iterrows():
  sentence = [row['palavra']]
  syllables = []
  modified_syllables = []
  chunks = []
  for syllable in row['silabas'].split('.'):
    modified_syllable = f'{syllables.count(syllable)}-{syllable}'
    syllables.append(syllable)
    modified_syllables.append(modified_syllable)
    sentence.append(modified_syllable)
  for i in range(2, len(modified_syllables)+1):
    for j in range(0, len(modified_syllables)):
      start = j
      end = j + i
      if (end <= len(modified_syllables)):
        chunk = ' '.join(syllables[start:end])
        sentence.append(f"{chunks.count(chunk)}_{chunk}")
        chunks.append(chunk)
  corpus_silabas_fast_text_cc.append(sentence)

In [ ]:
for k, v in enumerate(corpus_silabas_fast_text_cc[:3]):
  print(f'{k}: {v}')

0: ['aba', "0-'a", '0-ba', "0_'a ba"]
1: ['ababa', '0-a', "0-'ba", '0-ba', "0_a 'ba", "0_'ba ba", "0_a 'ba ba"]
2: ['abaçai', '0-a', '0-ba', '0-ça', "0-'i", '0_a ba', '0_ba ça', "0_ça 'i", '0_a ba ça', "0_ba ça 'i", "0_a ba ça 'i"]


In [ ]:
model_silabas_fast_text_cc = FastText(corpus_silabas_fast_text_cc, min_count=1, workers=8, window=10, sg=1, size=300, iter=30, min_n=2, max_n=2, callbacks=[epoch_logger])
print(model_silabas_fast_text_cc.wv.most_similar('simtetizando'))

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
Epoch #10 start
Epoch #10 end
Epoch #11 start
Epoch #11 end
Epoch #12 start
Epoch #12 end
Epoch #13 start
Epoch #13 end
Epoch #14 start
Epoch #14 end
Epoch #15 start
Epoch #15 end
Epoch #16 start
Epoch #16 end
Epoch #17 start
Epoch #17 end
Epoch #18 start
Epoch #18 end
Epoch #19 start
Epoch #19 end
Epoch #20 start
Epoch #20 end
Epoch #21 start
Epoch #21 end
Epoch #22 start
Epoch #22 end
Epoch #23 start
Epoch #23 end
Epoch #24 start
Epoch #24 end
Epoch #25 start
Epoch #25 end
Epoch #26 start
Epoch #26 end
Epoch #27 start
Epoch #27 end
Epoch #28 start
Epoch #28 end
Epoch #29 start
Epoch #29 end
[('sintetizando', 0.9506893157958984), ('simpatizando', 0.8880183100700378), ('sistematizando', 0.8691751956939697), ('s

In [ ]:
restrict_w2v(model_silabas_fast_text_cc, restricted_word_set)

In [ ]:
print('Most similar to jaguapita:')
for k, v in enumerate(model_silabas_fast_text_cc.wv.most_similar('jaguapita')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to gue:')
for k, v in enumerate(model_silabas_fast_text_cc.wv.most_similar('gue')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to grisalha:')
for k, v in enumerate(model_silabas_fast_text_cc.wv.most_similar('grisalha')[:5]):
  print(f'{v[0]}: {v[1]}')
print("")
print('Most similar to sintetizado:')
for k, v in enumerate(model_silabas_fast_text_cc.wv.most_similar('sintetizado')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to aba:')
for k, v in enumerate(model_silabas_fast_text_cc.wv.most_similar('aba')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bata:')
for k, v in enumerate(model_silabas_fast_text_cc.wv.most_similar('bata')[:5]):
  print(f'{v[0]}: {v[1]}')

print("")
print('Most similar to bantu:')
for k, v in enumerate(model_silabas_fast_text_cc.wv.most_similar('bantu')[:5]):
  print(f'{v[0]}: {v[1]}')

Most similar to jaguapita:
tagua: 0.8346182107925415
ipigua: 0.8198592066764832
apita: 0.8171899318695068
jagua: 0.7994146347045898
catagua: 0.7905172109603882

Most similar to gue:
guegue: 0.9749554395675659
ague: 0.9539045691490173
fugue: 0.8986433148384094
guegues: 0.8922747373580933
iogue: 0.8908430337905884

Most similar to grisalha:
grisalho: 0.9117481708526611
grisalhos: 0.8693745136260986
grilhao: 0.7669934630393982
salsaparrilha: 0.754426121711731
isaurinha: 0.7466573119163513

Most similar to sintetizado:
sintetizador: 0.9409784078598022
sintetizados: 0.9233722686767578
sintetizando: 0.9196770191192627
sintetiza: 0.913326621055603
sintetizadores: 0.8995703458786011

Most similar to aba:
ababa: 0.9696647524833679
abaiba: 0.93632572889328
baba: 0.9309819936752319
iba: 0.9142904281616211
ifba: 0.9099579453468323

Most similar to bata:
taba: 0.9562872648239136
baeta: 0.9385713338851929
tatajuba: 0.9149723052978516
futaba: 0.8978768587112427
itauba: 0.8906502723693848

Most simila

In [ ]:
print(model_silabas_fast_text_cc.wv.most_similar('simtetizando'))
print(model_silabas_fast_text_cc.wv.most_similar('facista'))
print(model_silabas_fast_text_cc.wv.most_similar('facistas'))
print(model_silabas_fast_text_cc.wv.most_similar('panda'))
print(model_silabas_fast_text_cc.wv.most_similar('pamda'))
print(model_silabas_fast_text_cc.wv.most_similar('caxorro'))
print(model_silabas_fast_text_cc.wv.most_similar('bantu'))

[('sintetizando', 0.9506893157958984), ('simpatizando', 0.8880183100700378), ('sistematizando', 0.8691751956939697), ('sintetizado', 0.8590971231460571), ('simpatizante', 0.854383647441864), ('optimizando', 0.8507772088050842), ('tematizando', 0.8504618406295776), ('batizando', 0.838541567325592), ('otimizando', 0.8343957662582397), ('mimetizando', 0.8173812031745911)]
[('cista', 0.9109761714935303), ('facistas', 0.8981961607933044), ('neofascista', 0.8448803424835205), ('pistacia', 0.8138492703437805), ('antifascista', 0.8093978762626648), ('publicista', 0.8079401254653931), ('ciclista', 0.8066648244857788), ('fatalista', 0.805901050567627), ('cita', 0.8007341027259827), ('pacifista', 0.7996814846992493)]
[('fascistas', 0.949663519859314), ('cistas', 0.92410808801651), ('neofascistas', 0.8949277400970459), ('neofascista', 0.8805069923400879), ('racistas', 0.8676984310150146), ('antifascistas', 0.8655707836151123), ('anti-fascistas', 0.858635425567627), ('antifascista', 0.8438457250595

In [ ]:
model_silabas_fast_text_cc.wv.save_word2vec_format("model_silabas_fast_text_cc.model", binary=False)
word2vec2tensor.word2vec2tensor("model_silabas_fast_text_cc.model", "tensor-model-silabas-fast-text-cc", False)